<a href="https://colab.research.google.com/github/satoicu/aes-rsa-crypt/blob/main/AES%26RSA_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.9/712.9 kB 51.4 MB/s eta 0:00:00


In [ ]:
!ssh-keygen

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:IJY5h1tSAmwHL9AreTeRFhBDhi+Bd2ovZ9cjoFug/1Q root@e59273823596
The key's randomart image is:
+---[RSA 3072]----+
|o=O=oo.          |
|+o=o**           |
| *.*X.+          |
|+ Bo+B .         |
| * =.oE.S        |
|. o =.o o        |
| . *.. . .       |
|  o.             |
|   ..            |
+----[SHA256]-----+


In [ ]:
!ls /root/.ssh

id_rsa	id_rsa.pub


In [ ]:
import json
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA

In [ ]:
# 暗号化するサンプルテキストを生成
sample={"name": "John Doe","age": 30,"email": "johndoe@example.com"}
message = json.dumps(sample)

In [ ]:
# サンプルテキストをbyte変換
data = message.encode('utf-8')

# 鍵の生成
key = get_random_bytes(32)

# AES暗号化
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)
iv = cipher.nonce

In [ ]:
# base64エンコード（byteからstringへ）
keyb64 = b64encode(key).decode('utf-8')
tagb64 = b64encode(tag).decode('utf-8')
ivb64 = b64encode(iv).decode('utf-8')
ciphertextb64 = b64encode(ciphertext).decode('utf-8')

In [ ]:
# base64で文字列化した情報を暗号化文（ciphertext)、復号化に必要な情報（keyinfo)に分割して保管
keyinfob64 = {"key":keyb64,
           "tag":tagb64,
           "iv":ivb64}
keyinfob64json = json.dumps(keyinfob64)

In [ ]:
ciphertextb64, keyinfob64json

('XRLW5GR6Dt29WP6raTsPZtuIlHxwHqsQSVnvf+O3n07ZcZtBOYnuqrpCdpZCWhze/N5JbE6DEgj4EGN3jWKE',
 '{"key": "uZcrBgJ6fKc0Bwp+dw522b7YzIPyOYwawecnLV2hucI=", "tag": "ZsBQsPCOge7GM9H1b5WCrQ==", "iv": "BqYipfEXUxd8kupIcsn89g=="}')

In [ ]:
# RSA暗号化のため公開鍵の読み込み
with open("/root/.ssh/id_rsa.pub", "rb") as f:
    public_key = RSA.import_key(f.read())

In [ ]:
# RSA暗号化のためbyteに変換
keyinfojson = keyinfob64json.encode('utf-8')

# RSA暗号化
rsa = PKCS1_OAEP.new(public_key)
cipher_keyinfo = rsa.encrypt(keyinfojson)

# byteをstringに変換
cipher_keyinfob64 = b64encode(cipher_keyinfo).decode('utf-8')

In [ ]:
cipher_keyinfob64

'yjQhfwiDwwSrHg2IcPD1ooNos5uJg5vcZTExhhypd07YQpbErkr/zV6fFDNzeewrHnm29+63Gb+ZiQDQf3vAWSSsdB/fsXEombu/NNFkDd56460W81ORM+B5ddoR8zJVdRUgjTQEsKV3o05xAqs+mEm632qocsytdra0q+I3vEynYE1iu8S5ZBiNBU72K2nk3kbgDP5csWKf4IjzxxJJ1ToAGvYAWqu0QtLH9GG0V9ayTUROEHYw7fVIdB0dpZGKdUzG1GSvpWDSGHWYygASQXG0CcqBoifVXUxWaoJ3i2fC7xkE9u5vbDRE1VxRowwwbz4Le/v0ONgs4QjWr+Hd8IWb6LmLO+XGvjIq8qcqbhDC1LUYPxny89Ctfdd01+Pbj11gvJJdtGHtf6G/AzY1haQ9IvRefK+5C2eNwMxGc3l623IdG5LsKfC+IVsOZb7MgcZxrSPbI9/YtbUFQtuL6AluVt5Sx7qO3Sab0Vms3Z5U1G71x9dDnBvX7CTAtKpc'

In [ ]:
#復号化して情報を取得するため暗号化前の情報を削除
del message, key, tag, iv

In [ ]:
#復号化のため秘密鍵の読み込み
with open("/root/.ssh/id_rsa", "rb") as f:
    private_key = RSA.import_key(f.read())

In [ ]:
# keyinfoのbyte変換
cipher_keyinfo = b64decode(cipher_keyinfob64.encode('utf-8'))

# 鍵情報のRSA復号化
decipher_rsa = PKCS1_OAEP.new(private_key)
keyinfojson = decipher_rsa.decrypt(cipher_keyinfo)

# byteをdict変換
keyinfo = json.loads(keyinfojson.decode('utf-8'))

In [ ]:
b64decode(keyinfo['key'].encode('utf-8'))

b"\xb9\x97+\x06\x02z|\xa74\x07\n~w\x0ev\xd9\xbe\xd8\xcc\x83\xf29\x8c\x1a\xc1\xe7'-]\xa1\xb9\xc2"

In [ ]:
# 鍵情報からkey,tag, ivを取り出す
key = b64decode(keyinfo['key'].encode('utf-8'))
tag = b64decode(keyinfo['tag'].encode('utf-8'))
iv = b64decode(keyinfo['iv'].encode('utf-8'))

# 暗号化文をbase64decodeする
ciphertext = b64decode(ciphertextb64.encode('utf-8'))

# AES復号化
decipher_aes = AES.new(key, AES.MODE_EAX, iv)
decrypted_text = decipher_aes.decrypt_and_verify(ciphertext, tag)

plaintext = decrypted_text.decode('utf-8')

In [ ]:
# 結果の表示
print("暗号化前のテキスト:", sample)
print("復号化後のテキスト:", plaintext)
print(message == plaintext)

暗号化前のテキスト: {'name': 'John Doe', 'age': 30, 'email': 'johndoe@example.com'}
復号化後のテキスト: {"name": "John Doe", "age": 30, "email": "johndoe@example.com"}
True
